# Pull location names from headlines

In [1]:
import os
import re
import unidecode

import pandas as pd
import geonamescache

In [2]:
def clean_text(txt):
    return unidecode.unidecode(txt)

In [3]:
# Read in data
headlines_file = os.path.join("data", "headlines.txt")

with open(headlines_file) as file:
    lines = [clean_text(line.strip()) for line in file]

In [4]:
len(lines)

650

In [5]:
[print(line) for line in lines[:20]];

Zika Outbreak Hits Miami
Could Zika Reach New York City?
First Case of Zika in Miami Beach
Mystery Virus Spreads in Recife, Brazil
Dallas man comes down with case of Zika
Trinidad confirms first Zika case
Zika Concerns are Spreading in Houston
Geneve Scientists Battle to Find Cure
The CDC in Atlanta is Growing Worried
Zika Infested Monkeys in Sao Paulo
Brownsville teen contracts Zika virus
Mosquito control efforts in St. Louis take new tactics with Zika threat
San Juan reports 1st U.S. Zika-related death amid outbreak
Flu outbreak in Galveston, Texas
Zika alert - Manila now threatened
Zika afflicts 7 in Iloilo City
New Los Angeles Hairstyle goes Viral
Louisiana Zika cases up to 26
Orlando volunteers aid Zika research
Zika infects pregnant woman in Cebu


**Potential problems**  
 - Locations at different levels in the taxonomy - Orlando/Florida/US
 - Multiple locations in headline
 - Split names - New York
 - Case (re.IGNORECASE)
 - Punctuation
 - Some aren't virus' i.e. Hairstyle goes Viral
 - Misspellings, non standard names
 

In [6]:
# Get location name lists
gc = geonamescache.GeonamesCache()

city_data = gc.get_cities()
cities = [city_data[city_id]['name'] for city_id in city_data]
cities = list(set(cities))  # ensure unique

countries = gc.get_countries()
countries = [country for country in gc.get_countries_by_names()]
countries = list(set(countries))

In [7]:
city_map = {city: clean_text(city) for city in cities}
country_map = {country: clean_text(country) for country in countries}

In [8]:
len(countries), len(cities)

(252, 23090)

In [9]:
# Get regex
city_bar_list = '|'.join(set(city_map.values()))
city_regex = re.compile(r'\b(' + r'{}'.format(city_bar_list) + r')\b', flags=re.IGNORECASE)

country_bar_list = '|'.join(set(country_map.values()))
country_regex = re.compile(r'\b(' + country_bar_list + r')\b', flags=re.IGNORECASE)

In [10]:
def find_location(line, location_regex):
    # Return the longest matching word from the regex
    try:
        match_list = location_regex.findall(line)
        current_best = max(match_list, key=len)
        if isinstance(current_best, str):
            return current_best
        else:
            return max(list(sum(match_list, ())), key=len)
    except:
        return None

In [11]:
# Create a dataframe
df = pd.DataFrame(lines, columns=['headline'])
df.head()

,headline
0,Zika Outbreak Hits Miami
1,Could Zika Reach New York City?
2,First Case of Zika in Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil"
4,Dallas man comes down with case of Zika


In [12]:
# Add in the city and country columns
df['city'] = df.apply(lambda x: find_location(x['headline'], city_regex), axis=1)
df['country'] = df.apply(lambda x: find_location(x['headline'], country_regex), axis=1)

In [13]:
df.head(20)

,headline,city,country
0,Zika Outbreak Hits Miami,Miami,None
1,Could Zika Reach New York City?,New York City,None
2,First Case of Zika in Miami Beach,Miami,None
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
4,Dallas man comes down with case of Zika,Dallas,None
5,Trinidad confirms first Zika case,Trinidad,None
6,Zika Concerns are Spreading in Houston,Houston,None
7,Geneve Scientists Battle to Find Cure,Geneve,None
8,The CDC in Atlanta is Growing Worried,Atlanta,None
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,None


In [15]:
# output_file = os.path.join("data", "headline_locations.parquet")
# df.to_parquet(output_file)

In [17]:
# import pickle
# pickle.dump(city_map, open("data/city_map.pkl", "wb"))
# pickle.dump(country_map, open("data/country_map.pkl", "wb"))